# Piano Phraser (ver. 0.5)

***

Powered by tegridy-tools: https://github.com/asigalov61/tegridy-tools

***

#### Project Los Angeles

#### Tegridy Code 2022

***

In [ ]:
#@title Install dependencies
!git clone https://github.com/asigalov61/tegridy-tools

In [ ]:
#@title Import modules
from collections import Counter
import tqdm

%cd /content/tegridy-tools/tegridy-tools/

import TMIDIX
import HaystackSearch

%cd /content/

In [ ]:
#@title Split Piano MIDI into phrases
full_path_to_MIDI_file = "/content/tegridy-tools/tegridy-tools/seed2.mid" #@param {type:"string"}

print('=' * 70)
print('Piano Phrasier')
print('=' * 70)

score = TMIDIX.midi2ms_score(open(full_path_to_MIDI_file, 'rb').read())

events_matrix = []

itrack = 1

while itrack < len(score):
    for event in score[itrack]:         
        if event[0] == 'note' and event[3] != 9:
            events_matrix.append(event)
    itrack += 1

# Sorting...
events_matrix.sort(key=lambda x: x[4], reverse=True)
events_matrix.sort(key=lambda x: x[1])

# recalculating timings
for e in events_matrix:
    e[1] = int(e[1] / 16)
    e[2] = int(e[2] / 32)

# final processing...

melody = []
melody_chords = []

pe = events_matrix[0]
for e in events_matrix:

    time = max(0, min(127, e[1]-pe[1]))
    dur = max(1, min(127, e[2]))

    ptc = max(1, min(127, e[4]))
    vel = max(1, min(127, e[5]))

    melody_chords.append([time, dur, ptc, vel])

    if time != 0:
      melody.append([time, dur, ptc, vel])

    pe = e

melody_pitches = []
melody_tones = []

for m in melody:
  melody_pitches.append(m[2])
  melody_tones.append(m[2] % 12)

melody_chorded = []
cho = []
for m in melody_chords:

  if m[0] == 0:
    cho.append(m)
  else:
    melody_chorded.append(cho)
    cho = []
    cho.append(m)

# =================================================================

def testAinB(A, B):
    n = len(A)
    return any(A == B[i:i + n] for i in range(len(B)-n + 1))

progressions = [Counter( (*melody_tones[i:i+size],) 
                         for i in range(0,len(melody_tones)-size+1)
                       ).most_common(1)[0]
                for size in tqdm.tqdm(range(len(melody_tones),3,-1))]

progs = []

for i in progressions:
  # print(list(i[0]))
  progs.append(list(i[0]))

phrases = []
plist = progs[0]

for t in progs:
  if testAinB(t, plist):
    pass
  else:
    phrases.append(t)

  plist = t

#print(phrases)
#print([len(y) for y in phrases])


phrases_f = []
gindexes = [0]

for phr in phrases:

  idx = 0
  index = []
  gindex = [0]
  for i in range(len(melody_tones)):

    id = HaystackSearch.HaystackSearch(phr, melody_tones[idx:])
    if id != -1:
      gindex.append(gindex[-1]+id)
      # print(gindex[-1])
      gindexes.append(gindex[-1])
      index.append(id)
      
      # print(index[-1])
      idx = sum(index)+len(index)
    else:
      break
  phrases_f.append([phr, gindex[1:]])
gindexes.append(len(melody_tones))
gindexes.sort()

#print(phrases_f)
#print(gindexes)

# =================================================================

song1 = []

for i in range(len(gindexes)-1):
  song = []
  for j in range(gindexes[i]+1, gindexes[i+1]):
    song.extend(melody_chorded[j])

  song1.append(song)

song2 = []

for s in song1:

  song2.extend(s)
  song2.append([127, 127, 0, 0])
  song2.append([127, 127, 0, 0])

out1 = song2

if len(out1) != 0:
    
    song = out1
    song_f = []
    time = 0
    dur = 0
    vel = 0
    pitch = 0
    channel = 0
    
    for s in song:


        channel = 0



        vel = s[3]
        
      

        time += s[0] * 16
            
        dur = s[1] * 32
        
        pitch = s[2]
                                  
        song_f.append(['note', time, dur, channel, pitch, vel ])

    detailed_stats = TMIDIX.Tegridy_SONG_to_MIDI_Converter(song_f,
                                                        output_signature = 'Piano Phraser',  
                                                        output_file_name = '/content/Piano-Phraser-Composition', 
                                                        track_name='Project Los Angeles',
                                                        list_of_MIDI_patches=[0, 24, 32, 40, 42, 46, 56, 71, 73, 0, 53, 0, 0, 0, 0, 0],
                                                        number_of_ticks_per_quarter=500)

    print('Done!')


# Congrats! You did it! :)